In [1]:
import numpy
import tensorflow as tf

from model_builder import model_builder, relabel, class_merger, balancer

import model_builder
import tools_keras

from itertools import product

In [2]:
specs = {
    'tensor': (
        '../urbangrammar_samba/'
        'spatial_signatures/chips/sample.npz'
    ),
    'folder': "../urbangrammar_samba/spatial_signatures/ai/nw/",
}

In [3]:
data = numpy.load(specs["tensor"], allow_pickle=True)

In [4]:
chips = data["chips"]
labels = data["labels"]

Shuffle data to ensure full randomness before splitting to test, validation and secret.

In [5]:
numpy.random.seed(0)

shuffled_idx = numpy.arange(0, chips.shape[0])
numpy.random.shuffle(shuffled_idx)

chips = chips[shuffled_idx]
labels = labels[shuffled_idx]

Merge groups

In [6]:
group_mapping = [
    ['9_0', '9_1', '9_2', '9_4', '9_5', '2_0', '2_1', '2_2'],
    ['1_0', '3_0', '5_0', '6_0', '8_0'],
    ['0_0', '4_0', '7_0']
]
labels = class_merger(labels, group_mapping)

# If you don't want to merge groups, use `relabel` to ensure your labels are in the proper format
# labels = relabel(labels)

Drop chips from excessive classes

In [7]:
mask = balancer(labels, max_ratio=10)

Total number of selected chips: 18841 out of 65042
Counts:
 {0: 907, 1: 8864, 2: 9070}


In [8]:
labels = labels[mask]
chips = chips[mask]

In [9]:
chips = chips / (chips.max() / 255.0)

In [13]:
centres = labels == 0
labels_centres = labels[centres.flatten()]
chips_centres = chips[centres.flatten()]
labels_other = labels[~centres.flatten()]
chips_other = chips[~centres.flatten()]

In [16]:
centres = tf.data.Dataset.from_tensor_slices((chips_centres, labels_centres))

2021-12-16 15:23:50.622056: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 120 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:21:00.0, compute capability: 7.5


In [21]:
centres = centres.map(lambda image, label: (tf.image.random_flip_left_right(image), label)).repeat(2)

In [22]:
len([c for c in centres])

1814

In [ ]:
from itertools import chain

merged_generator = chain(train_generator_cats, train_generator_others)